# From Delta Lake to Amazon SageMaker

[Delta Lake](https://delta.io/) is a common open-source framework used for storing data in Lakehouse architectures.

In this sample we demonstrate how to integrate Delta Tables with Amazon SageMaker for performing data exploration, ingestion, processing, training, and hosting for Machine Learning.

---

## 1 - Data Exploration and Visualization

***Use Kernel: SparkMagic (PySpark) for running this notebook***

In this notebook, we will perform some Exploratory Data Analysis (EDA) over our Delta Tables with the two connection methods explained in the previous notebook.


In [2]:
import sagemaker
sagemaker.__version__

'2.107.0'

In [3]:
import numpy as np
import pandas as pd
import boto3

In [4]:
# S3 bucket for saving processing job outputs
sm_session = sagemaker.Session()
bucket = sm_session.default_bucket()
region = sm_session.boto_region_name

sm_client = boto3.client('sagemaker')
iam_role = sagemaker.get_execution_role()

print('Default bucket: '+bucket)

Default bucket: sagemaker-eu-west-1-889960878219


----
### Option 1: Reading from a Delta Table stored in Amazon S3 via Spark Session and Context

In [5]:
# Import pyspark and build Spark session
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext

In [6]:
# Build list of packages entries using Maven coordinates (groupId:artifactId:version)
pkg_list = []
pkg_list.append("io.delta:delta-core_2.12:1.1.0")
pkg_list.append("org.apache.hadoop:hadoop-aws:3.2.2")

packages=(",".join(pkg_list))
print('packages: '+packages)

packages: io.delta:delta-core_2.12:1.1.0,org.apache.hadoop:hadoop-aws:3.2.2


In [7]:
# Instantiate Spark via builder
# Note: we use the `ContainerCredentialsProvider` to give us access to underlying IAM role permissions

spark = (SparkSession
    .builder
    .appName("PySparkApp") 
    .config("spark.jars.packages", packages) 
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
    .config("fs.s3a.aws.credentials.provider",'com.amazonaws.auth.ContainerCredentialsProvider') 
    .getOrCreate())

sc = spark.sparkContext

print('Spark version: '+str(sc.version))

:: loading settings :: url = jar:file:/opt/conda/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1dcb8ee3-30c5-465a-abb4-807835d16ea3;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 708ms :: artifacts dl 96ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	--------

Spark version: 3.2.0


In [8]:
s3a_delta_table_uri=f's3a://{bucket}/delta_to_sagemaker/delta_format/'
print(s3a_delta_table_uri)

s3a://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/delta_format/


In [9]:
# Create SQL command inserting the S3 path location

sql_cmd = f'SELECT * FROM delta.`{s3a_delta_table_uri}` ORDER BY medv'
print(f'SQL command: {sql_cmd}')

SQL command: SELECT * FROM delta.`s3a://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/delta_format/` ORDER BY medv


In [ ]:
# Execute SQL command which returns dataframe

sql_results = spark.sql(sql_cmd)
print(type(sql_results))

sql_results.show(10)

22/08/31 13:16:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


(TBC).....

----
### Option 2: Reading from an external Delta Table via Delta Sharing

In [ ]:
profile_file = "https://raw.githubusercontent.com/delta-io/delta-sharing/main/examples/open-datasets.share"
!wget {profile_file} -P ./notebooks/

In [ ]:
!cat ./notebooks/open-datasets.share

In [ ]:
sample_profile_file_url = sagemaker.Session().upload_data(
    "./notebooks/open-datasets.share", bucket=bucket, key_prefix="delta_to_sagemaker/delta_sharing/profile"
)

print(sample_profile_file_url)

In [ ]:
# Create a SharingClient
import delta_sharing

client = delta_sharing.SharingClient(sample_profile_file_url)
table_url = profile_file + "#delta_sharing.default.boston-housing"

In [ ]:
# Load the table as a Pandas DataFrame
print('Loading boston-housing table from Delta Lake')
train_data = delta_sharing.load_as_spark(table_url)
print(f'Train data shape: {train_data.shape}')